In [ ]:
!pip install ktrain

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from ktrain import text
import pandas as pd
import numpy as np
import ktrain
import nltk
import re

In [3]:
def get_data(url):
    f = open(url, 'r', encoding = "UTF-8")
    text = f.read()
    text_array = text.split('\n')
    text_dict = {'text':[], 'emotion':[]}
    emotion_dict = {"anger": 1, "joy":2, "fear":3, "sadness":4, "love":5, "surprise":6}
    for sentence in text_array:
        if ';' in sentence:
            a = sentence.split(';')
            text_dict['text'].append(a[0])
            text_dict['emotion'].append(a[1])
    df = pd.DataFrame(text_dict)
    return(df)

In [4]:
train = get_data('train.txt')

In [5]:
print("Shape of train is", train.shape)
train.head()

Shape of train is (16000, 2)


,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [6]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df= train, 
                                                                      text_column = 'text',
                                                                      label_columns = 'emotion',
                                                                      maxlen = 50,
                                                                      preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
model = text.text_classifier(name= 'bert',
                             train_data= (X_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 50
done.


In [8]:
# Get Learning Rate
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 32)

In [9]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
450/450 [==============================] - 251s 558ms/step - loss: 0.8008 - accuracy: 0.6956 - val_loss: 0.2308 - val_accuracy: 0.9150


In [10]:
predictor = ktrain.get_predictor(learner.model, preproc= preprocess)

In [11]:
data = ["I am pretty sad and I don't have anyone to talk to.", 
        "I am great actually. Very happy with my life for now",
        "I am really scared of what is going to happen to me now"]

In [12]:
predictor.predict(data)

['sadness', 'joy', 'fear']

In [13]:
data = ["Not great. I feel suicidal and I have no one to talk to.",
        "Happy, optimistic, inspired, enthusiastic, upbeat, silly, joyful!!",
        "I feel tired, sore, and lonely. I just wan't somebody to hold.",
        "I got in trouble at work and I feel this way. like i'm going to lose my job tomorrow"]

In [14]:
predictor.predict(data)

['sadness', 'joy', 'sadness', 'sadness']

In [15]:
test = get_data('test.txt')
test.head()

,text,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [16]:
y_t = np.array(test['emotion'])
y_t

array(['sadness', 'sadness', 'sadness', ..., 'joy', 'joy', 'fear'],
      dtype=object)

In [17]:
data = np.array(test['text'])

In [18]:
y_pred = predictor.predict(data)

In [19]:
y_pred = np.array(y_pred)
y_pred

array(['sadness', 'sadness', 'sadness', ..., 'joy', 'joy', 'fear'],
      dtype='<U8')

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
print("Accuracy score for test data is:")
print(accuracy_score(y_t, y_pred))
print("\nConfusion Matrix for test data is:")
print(confusion_matrix(y_t, y_pred))

Accuracy score for test data is:
0.91

Confusion Matrix for test data is:
[[247   5   5   2  16   0]
 [  7 203   1   0   5   8]
 [  2   0 646  36   9   2]
 [  2   0  19 137   1   0]
 [ 14   9  12   2 544   0]
 [  2  14   6   0   1  43]]


In [21]:
data = ["I have really become used to this cycle where I stay happy for 14-15 days and then spend the next 5-6 days feeling worthless and feeling like doing nothing at all.These emotions are not clear, they are not the primary emotions that I face because I don’t have a clear picture of what I am feeling.I haven’t felt love, I have stopped feeling any kind of emotion and I have become numb, completely numb.This feeling of emptiness, like a void which cannot be filled now.Even when the lockdown is not there, I feel like I am in lockdown.I guess this 3 weeks cycle is my routine now until things get better and life gets back to what it was."]

In [22]:
predictor.predict(data)

['sadness']